# TensorFlow and TensorBoard with Regularization



## Purpose

The purpose of this lab is threefold.  

1.   to review using `TensorFlow` and `TensorBoard` for modeling and evaluation with neural networks
2.   to review using data science pipelines and cross-validation with neural networks
3.   to review using `TensorFlow` for neural network regularization

We'll be continuting our investigation of the canonical [Titanic Data Set](https://www.kaggle.com/competitions/titanic/overview) that we began [previously](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

## The Titanic

### The Titanic and it's data



RMS Titanic was a British passenger liner built by Harland and Wolf and operated by the White Star Line. It sank in the North Atlantic Ocean in the early morning hours of 15 April 1912, after striking an iceberg during her maiden voyage from Southampton, England to New York City, USA.

Of the estimated 2,224 passengers and crew aboard, more than 1,500 died, making the sinking one of modern history's deadliest peacetime commercial marine disasters. 

Though there were about 2,224 passengers and crew members, we are given data of about 1,300 passengers. Out of these 1,300 passengers details, about 900 data is used for training purpose and remaining 400 is used for test purpose. The test data has had the survived column removed and we'll use neural networks to predict whether the passengers in the test data survived or not. Both training and test data are not perfectly clean as we'll see.

Below is a picture of the Titanic Museum in Belfast, Northern Ireland.

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://upload.wikimedia.org/wikipedia/commons/c/c0/Titanic_Belfast_HDR.jpg", width=400, height=400)

### Data Dictionary

*   *Survival* : 0 = No, 1 = Yes
*   *Pclass* : A proxy for socio-economic status (SES)
  *   1st = Upper
  *   2nd = Middle
  *   3rd = Lower
*   *sibsp* : The number of siblings / spouses aboard the Titanic
  *   Sibling = brother, sister, stepbrother, stepsister
  *   Spouse = husband, wife (mistresses and fiancés were ignored)
*   *parch* : The # of parents / children aboard the Titanic
  *   Parent = mother, father
  *   Child = daughter, son, stepdaughter, stepson
  *   Some children travelled only with a nanny, therefore *parch*=0 for them.
*   *Ticket* : Ticket number
*   *Fare* : Passenger fare (British pounds)
*   *Cabin* : Cabin number embarked
*   *Embarked* : Port of Embarkation
  *   C = Cherbourg (now Cherbourg-en-Cotentin), France
  *   Q = Queenstown (now Cobh), Ireland
  *   S = Southampton, England
*   *Name*, *Sex*, *Age* (years) are all self-explanatory

## Libraries and the Data



### Importing libraries

In [4]:
# Load the germane libraries

import pandas as pd
import numpy as np
import seaborn as sns 
from pandas._libs.tslibs import timestamps
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras 
from keras import models
from sklearn.impute import SimpleImputer
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.losses import binary_crossentropy
from sklearn.model_selection import GridSearchCV
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier

# Load the TensorBoard notebook extension and related libraries
%load_ext tensorboard
import datetime

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Loading the data

In [5]:
# Load the data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# We need to do this for when we mamke our predictions from the test data at the end
ids = test[['PassengerId']]

## EDA and Preprocessing

### Exploratory Data Analysis

You have already performed EDA on this data set. Look back on what you did before or see [here](https://github.com/learn-co-curriculum/enterprise-paired-nn-eval).

Of course, feel free to re-run what you have done before or try out some other EDA as you find useful.

### Preprocessing

Let's do the same prepricessing as before.

In [6]:
# Performing preprocessing on the train and test data will be more effecient if we combine the two date sets.
combined = pd.concat([train, test], axis=0, sort=False)

#Age column
combined['Age'].fillna(combined['Age'].median(),inplace=True) # Age

# Embarked column
combined['Embarked'].fillna(combined['Embarked'].value_counts().index[0], inplace=True) # Embarked
combined['Fare'].fillna(combined['Fare'].median(),inplace=True)

# Class column
d = {1:'1st',2:'2nd',3:'3rd'} #Pclass
combined['Pclass'] = combined['Pclass'].map(d) #Pclass

# Making Age into adult (1) and child (0)
combined['Child'] = combined['Age'].apply(lambda age: 1 if age>=18 else 0) 

# Break up the string that has the title and names
combined['Title'] = combined['Name'].str.split('.').str.get(0)  # output : 'Futrelle, Mrs'
combined['Title'] = combined['Title'].str.split(',').str.get(1) # output : 'Mrs '
combined['Title'] = combined['Title'].str.strip()               # output : 'Mrs'
combined.groupby('Title').count()

# Replace the French titles with Enlgish
french_titles = ['Don', 'Dona', 'Mme', 'Ms', 'Mra','Mlle']
english_titles = ['Mr', 'Mrs','Mrs','Mrs','Mrs','Miss']
for i in range(len(french_titles)):
    for j in range(len(english_titles)):
        if i == j:
            combined['Title'] = combined['Title'].str.replace(french_titles[i],english_titles[j])

# Seperate the titles into "major" and "others", the latter would be, e.g., Reverend
major_titles = ['Mr','Mrs','Miss','Master']
combined['Title'] = combined['Title'].apply(lambda title: title if title in major_titles else 'Others')

#Dropping the Irrelevant Columns
combined.drop(['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)

# Getting Dummy Variables and Dropping the Original Categorical Variables
categorical_vars = combined[['Pclass','Sex','Embarked','Title','Child']] # Get Dummies of Categorical Variables
dummies = pd.get_dummies(categorical_vars,drop_first=True)
combined = combined.drop(['Pclass','Sex','Embarked','Title','Child'],axis=1)
combined = pd.concat([combined, dummies],axis=1)

# Separating the data back into train and test sets
test = combined[combined['Survived'].isnull()].drop(['Survived'],axis=1)
train = combined[combined['Survived'].notnull()]

# Training
X_train = train.drop(['Survived'],axis=1)
y_train = train['Survived']

# Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
test = sc.fit_transform(test)

## Neural Network Model

### Building the model

#### Define the model as a pipeline

Let's use the data science pipeline for our neural network model.

As you are now using regularization to guard against high variance, i.e. overfitting the data, in the definition of the model below include *dropout* and/or *l2* regularization. Also, feel free to experiment with different activation functions.

In [36]:
# It will help to define our model in terms of a pipeline
def build_classifier(optimizer):
# insert Sequential and layers here
    classifier = Sequential()
    # use classifer.add() to add layers
    classifier.add(Dense(units=14, activation='relu', input_dim=14, kernel_regularizer=l2(l2=0.05)))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(units=1, activation='sigmoid'))
    classifier.compile(loss='binary_crossentropy', metrics=['accuracy'])
    # use classifer.compile() as your last line of the definition; use loss='binary_crossentropy',metrics=['accuracy']
    return classifier

#### Use grid search to find help you tune the parameters

You can play with optimizers, epochs, and batch sizes. The ones that we're suggesting are not necessarily the best.

In [41]:
# Grid Search
classifier = KerasClassifier(build_fn = build_classifier)
param_grid = dict(optimizer = ['Adam'],
                  epochs=[10, 20, 50],
                  batch_size=[16, 25, 32])
grid = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

Epoch 1/10
45/45 [==============================] - 0s 2ms/step - loss: 1.2609 - accuracy: 0.6152
Epoch 2/10
45/45 [==============================] - 0s 4ms/step - loss: 1.1309 - accuracy: 0.6419
Epoch 3/10
45/45 [==============================] - 0s 2ms/step - loss: 0.9848 - accuracy: 0.6896
Epoch 4/10
45/45 [==============================] - 0s 2ms/step - loss: 0.8746 - accuracy: 0.7374
Epoch 5/10
45/45 [==============================] - 0s 2ms/step - loss: 0.7881 - accuracy: 0.7528
Epoch 6/10
45/45 [==============================] - 0s 667us/step - loss: 0.7117 - accuracy: 0.7697
Epoch 7/10
45/45 [==============================] - 0s 1ms/step - loss: 0.6627 - accuracy: 0.7781
Epoch 8/10
45/45 [==============================] - 0s 977us/step - loss: 0.6200 - accuracy: 0.7935
Epoch 9/10
45/45 [==============================] - 0s 955us/step - loss: 0.5792 - accuracy: 0.8174
Epoch 10/10
45/45 [==============================] - 0s 666us/step - loss: 0.5667 - accuracy: 0.8048
Epoch 1/10


45/45 [==============================] - 0s 1ms/step - loss: 0.5406 - accuracy: 0.8191
Epoch 14/20
45/45 [==============================] - 0s 889us/step - loss: 0.5255 - accuracy: 0.8177
Epoch 15/20
45/45 [==============================] - 0s 892us/step - loss: 0.5107 - accuracy: 0.8205
Epoch 16/20
45/45 [==============================] - 0s 889us/step - loss: 0.5064 - accuracy: 0.8191
Epoch 17/20
45/45 [==============================] - 0s 822us/step - loss: 0.4920 - accuracy: 0.8219
Epoch 18/20
45/45 [==============================] - 0s 1ms/step - loss: 0.4897 - accuracy: 0.8233
Epoch 19/20
45/45 [==============================] - 0s 802us/step - loss: 0.4814 - accuracy: 0.8233
Epoch 20/20
45/45 [==============================] - 0s 711us/step - loss: 0.4701 - accuracy: 0.8303
Epoch 1/20
45/45 [==============================] - 0s 4ms/step - loss: 1.3733 - accuracy: 0.5091
Epoch 2/20
45/45 [==============================] - 0s 2ms/step - loss: 1.1615 - accuracy: 0.5792
Epoch 3/20
4

45/45 [==============================] - 0s 911us/step - loss: 0.5043 - accuracy: 0.8315
Epoch 15/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4893 - accuracy: 0.8188
Epoch 16/50
45/45 [==============================] - 0s 956us/step - loss: 0.5000 - accuracy: 0.8202
Epoch 17/50
45/45 [==============================] - 0s 845us/step - loss: 0.4751 - accuracy: 0.8202
Epoch 18/50
45/45 [==============================] - 0s 733us/step - loss: 0.4811 - accuracy: 0.8202
Epoch 19/50
45/45 [==============================] - 0s 867us/step - loss: 0.4728 - accuracy: 0.8202
Epoch 20/50
45/45 [==============================] - 0s 667us/step - loss: 0.4658 - accuracy: 0.8301
Epoch 21/50
45/45 [==============================] - 0s 711us/step - loss: 0.4677 - accuracy: 0.8244
Epoch 22/50
45/45 [==============================] - 0s 867us/step - loss: 0.4639 - accuracy: 0.8287
Epoch 23/50
45/45 [==============================] - 0s 800us/step - loss: 0.4643 - accuracy: 0.8202
Epoc

45/45 [==============================] - 0s 1ms/step - loss: 0.4369 - accuracy: 0.8275
Epoch 47/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4367 - accuracy: 0.8261
Epoch 48/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4419 - accuracy: 0.8317
Epoch 49/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4419 - accuracy: 0.8317
Epoch 50/50
45/45 [==============================] - 0s 956us/step - loss: 0.4363 - accuracy: 0.8219
Epoch 1/50
45/45 [==============================] - 0s 442us/step - loss: 1.4630 - accuracy: 0.4025
Epoch 2/50
45/45 [==============================] - 0s 667us/step - loss: 1.2400 - accuracy: 0.5526
Epoch 3/50
45/45 [==============================] - 0s 831us/step - loss: 1.0708 - accuracy: 0.6858
Epoch 4/50
45/45 [==============================] - 0s 844us/step - loss: 0.9572 - accuracy: 0.7209
Epoch 5/50
45/45 [==============================] - 0s 759us/step - loss: 0.8596 - accuracy: 0.7349
Epoch 6/50
45/4

45/45 [==============================] - 0s 1ms/step - loss: 0.4523 - accuracy: 0.8429
Epoch 28/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4525 - accuracy: 0.8275
Epoch 29/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4463 - accuracy: 0.8345
Epoch 30/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4491 - accuracy: 0.8331
Epoch 31/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4314 - accuracy: 0.8415
Epoch 32/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4424 - accuracy: 0.8387
Epoch 33/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4421 - accuracy: 0.8317
Epoch 34/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4360 - accuracy: 0.8401
Epoch 35/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4407 - accuracy: 0.8289
Epoch 36/50
45/45 [==============================] - 0s 1ms/step - loss: 0.4336 - accuracy: 0.8359
Epoch 37/50
45/45 [===

...

23/23 [==============================] - 0s 2ms/step - loss: 0.8928 - accuracy: 0.7041
Epoch 8/20
23/23 [==============================] - 0s 2ms/step - loss: 0.8328 - accuracy: 0.7363
Epoch 9/20
23/23 [==============================] - 0s 1ms/step - loss: 0.7950 - accuracy: 0.7489
Epoch 10/20
23/23 [==============================] - 0s 1ms/step - loss: 0.7461 - accuracy: 0.7616
Epoch 11/20
23/23 [==============================] - 0s 1ms/step - loss: 0.7100 - accuracy: 0.7952
Epoch 12/20
23/23 [==============================] - 0s 1ms/step - loss: 0.6707 - accuracy: 0.7952
Epoch 13/20
23/23 [==============================] - 0s 1ms/step - loss: 0.6490 - accuracy: 0.7980
Epoch 14/20
23/23 [==============================] - 0s 2ms/step - loss: 0.6254 - accuracy: 0.8149
Epoch 15/20
23/23 [==============================] - 0s 2ms/step - loss: 0.5914 - accuracy: 0.8079
Epoch 16/20
23/23 [==============================] - 0s 2ms/step - loss: 0.5872 - accuracy: 0.8121
Epoch 17/20
23/23 [=====

Epoch 30/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4658 - accuracy: 0.8132
Epoch 31/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4689 - accuracy: 0.8357
Epoch 32/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4711 - accuracy: 0.8272
Epoch 33/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4825 - accuracy: 0.8216
Epoch 34/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4663 - accuracy: 0.8202
Epoch 35/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4732 - accuracy: 0.8160
Epoch 36/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4748 - accuracy: 0.8132
Epoch 37/50
23/23 [==============================] - 0s 956us/step - loss: 0.4566 - accuracy: 0.8301
Epoch 38/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4672 - accuracy: 0.8216
Epoch 39/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4665 - accuracy: 0.8118
Epoch 40

23/23 [==============================] - 0s 2ms/step - loss: 0.6658 - accuracy: 0.8177
Epoch 13/50
23/23 [==============================] - 0s 913us/step - loss: 0.6500 - accuracy: 0.7952
Epoch 14/50
23/23 [==============================] - 0s 2ms/step - loss: 0.6211 - accuracy: 0.8065
Epoch 15/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5914 - accuracy: 0.8135
Epoch 16/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5834 - accuracy: 0.8177
Epoch 17/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5708 - accuracy: 0.8163
Epoch 18/50
23/23 [==============================] - 0s 1ms/step - loss: 0.5547 - accuracy: 0.8121
Epoch 19/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5480 - accuracy: 0.8135
Epoch 20/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5331 - accuracy: 0.8219
Epoch 21/50
23/23 [==============================] - 0s 1000us/step - loss: 0.5189 - accuracy: 0.8191
Epoch 22/50
23/23

23/23 [==============================] - 0s 1ms/step - loss: 0.4383 - accuracy: 0.8373
Epoch 45/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4399 - accuracy: 0.8387
Epoch 46/50
23/23 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.8373
Epoch 47/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4399 - accuracy: 0.8373
Epoch 48/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4434 - accuracy: 0.8401
Epoch 49/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4447 - accuracy: 0.8345
Epoch 50/50
23/23 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.8317
Epoch 1/50
23/23 [==============================] - 0s 652us/step - loss: 1.5631 - accuracy: 0.3899
Epoch 2/50
23/23 [==============================] - 0s 652us/step - loss: 1.3929 - accuracy: 0.4951
Epoch 3/50
23/23 [==============================] - 0s 783us/step - loss: 1.2850 - accuracy: 0.5161
Epoch 4/50
23/23 [=

36/36 [==============================] - 0s 1ms/step - loss: 0.4666 - accuracy: 0.8238
Epoch 27/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4666 - accuracy: 0.8215
Epoch 28/50
36/36 [==============================] - 0s 805us/step - loss: 0.4648 - accuracy: 0.8204
Epoch 29/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4633 - accuracy: 0.8272
Epoch 30/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4729 - accuracy: 0.8272
Epoch 31/50
36/36 [==============================] - 0s 750us/step - loss: 0.4677 - accuracy: 0.8148
Epoch 32/50
36/36 [==============================] - 0s 2ms/step - loss: 0.4635 - accuracy: 0.8272
Epoch 33/50
36/36 [==============================] - 0s 833us/step - loss: 0.4681 - accuracy: 0.8238
Epoch 34/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4616 - accuracy: 0.8126
Epoch 35/50
36/36 [==============================] - 0s 1ms/step - loss: 0.4634 - accuracy: 0.8126
Epoch 36/50
36/3

In [42]:
best_accuracy

0.819295712761283

#### `TensorBoard`

`TensorBoard` is `TensorFlow`'s visualization toolkit. It is a dashboard that provides visualization and tooling that is needed for machine learning experimentation. The code immediately below will allow us to use TensorBoard.

N.B. When we loaded the libraries, we loaded the TensorBoard notebook extension. (It is the last line of code in the first code chunk.)

In [37]:
# Clear out any prior log data.
!rm -rf logs
# Be careful not to run this command if already have trained your model and you want to use TensorBoard.

# Sets up a timestamped log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(log_dir)


# The callback function, which will be called in the fit()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#### Fitting the optimal model and evaluating with `TensorBoaard`

Define the early stopping callback. Use your best values from grid serarch with `KerasClassifer` and finally fit the model.

In [38]:
# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           verbose=1, patience=5,
                           mode='min')

# Using KerasClassifier
classifier = KerasClassifier(build_fn = build_classifier,
                             optimizer=best_parameters['optimizer'],
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

# Fit the model with the tensorboard_callback
classifier.fit(X_train,
               y_train,
               verbose=1,
               callbacks=[early_stop, tensorboard_callback])


# Warning: If verbose = 0 (silent) or 2 (one line per epoch), then on TensorBoard's Graphs tab there will be an error.
# The other tabs in TensorBoard will still be function, but if you want the graphs then verbose needs to be 1 (progress bar).

Epoch 1/50
56/56 [==============================] - 0s 6ms/step - loss: 1.4974 - accuracy: 0.3827
Epoch 2/50
56/56 [==============================] - 0s 3ms/step - loss: 1.2030 - accuracy: 0.5365
Epoch 3/50
56/56 [==============================] - 0s 2ms/step - loss: 0.9988 - accuracy: 0.6779
Epoch 4/50
56/56 [==============================] - 0s 2ms/step - loss: 0.8538 - accuracy: 0.7419
Epoch 5/50
56/56 [==============================] - 0s 2ms/step - loss: 0.7388 - accuracy: 0.7912
Epoch 6/50
56/56 [==============================] - 0s 964us/step - loss: 0.6698 - accuracy: 0.7924
Epoch 7/50
56/56 [==============================] - 0s 2ms/step - loss: 0.6079 - accuracy: 0.8103
Epoch 8/50
56/56 [==============================] - 0s 2ms/step - loss: 0.5830 - accuracy: 0.8058
Epoch 9/50
56/56 [==============================] - 0s 2ms/step - loss: 0.5434 - accuracy: 0.8047
Epoch 10/50
56/56 [==============================] - 0s 2ms/step - loss: 0.5399 - accuracy: 0.8238
Epoch 11/50
56/56

56/56 [==============================] - 0s 3ms/step - loss: 0.4631 - accuracy: 0.8227
Epoch 27/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4525 - accuracy: 0.8283
Epoch 28/50
56/56 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.8272
Epoch 29/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4609 - accuracy: 0.8272
Epoch 30/50
56/56 [==============================] - 0s 1ms/step - loss: 0.4489 - accuracy: 0.8272
Epoch 31/50
56/56 [==============================] - 0s 930us/step - loss: 0.4503 - accuracy: 0.8316
Epoch 32/50
56/56 [==============================] - 0s 892us/step - loss: 0.4493 - accuracy: 0.8215
Epoch 33/50
56/56 [==============================] - 0s 1ms/step - loss: 0.4621 - accuracy: 0.8272
Epoch 34/50
56/56 [==============================] - 0s 2ms/step - loss: 0.4553 - accuracy: 0.8137
Epoch 35/50
56/56 [==============================] - 0s 3ms/step - loss: 0.4558 - accuracy: 0.8215
Epoch 36/50
56/56 

In [39]:
# Call TensorBoard within SaturnCloud [Comment this out if you are not in SaturnCloud]
import os
print(f"https://{os.getenv('SATURN_JUPYTER_BASE_DOMAIN')}/proxy/8000/")
%tensorboard --logdir logs/fit --port 8000 --bind_all
# This will generate a hyperlink. Click on that to open TensorBoard!
# (You'll see a 404 error below the link, just ignore that.)

# Call TensorBoard [Not in SaturnCloud]
# Uncomment the next time if you are not in SC
# %tensorboard --logdir logs/fit

https://None/proxy/8000/


Reusing TensorBoard on port 8000 (pid 25004), started 0:07:17 ago. (Use '!kill 25004' to kill it.)

#### Results and Predictions

Calculate the predictions, save them as a csv, and print them.

In [40]:
# Your code here (use more cells if you need to)
preds = classifier.predict(test)
results = ids.assign(Survived=preds)
results['Survived'] = results['Survived'].apply(lambda row: 1 if row > 0.5 else 0)
results.to_csv('titanic_submission.csv',index=False)
results.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


Continue to tweak your model until you are happy with the results based on model evaluation.

## Conclusion

Now that you have the `TensorBoard` to help you look at your model, you can better understand how to tweak your model.

How do your predictions compare to what you did last time?

Remember that your "fancier" model may be less accurate... but that is okay if that is the case since we're trying to guard against variance with regularization techniques.